In [2]:
import xarray as xr

In [3]:
feat_cols = ['TOT_NUM_CONC',
 'TOT_MASS_CONC',
 'pmc_SO4',
 'pmc_NO3',
 'pmc_Cl',
 'pmc_NH4',
 'pmc_ARO1',
 'pmc_ARO2',
 'pmc_ALK1',
 'pmc_OLE1',
 'pmc_API1',
 'pmc_API2',
 'pmc_LIM1',
 'pmc_LIM2',
 'pmc_OC',
 'pmc_BC',
 'pmc_H2O',
 'TEMPERATURE',
 'REL_HUMID',
 'ALT',
 'Z',
#  'XLAT',
#  'XLONG',
 'h2so4',
 'hno3',
 'hcl',
 'nh3',
 'no',
 'no2',
 'no3',
 'n2o5',
 'hono',
 'hno4',
 'o3',
 'o1d',
 'O3P',
 'oh',
 'ho2',
 'h2o2',
 'co',
 'so2',
 'ch4',
 'c2h6',
 'ch3o2',
 'ethp',
 'hcho',
 'ch3oh',
 'ANOL',
 'ch3ooh',
 'ETHOOH',
 'ald2',
 'hcooh',
 'RCOOH',
 'c2o3',
 'pan',
 'aro1',
 'aro2',
 'alk1',
 'ole1',
 'api1',
 'api2',
 'lim1',
 'lim2',
 'par',
 'AONE',
 'mgly',
 'eth',
 'OLET',
 'OLEI',
 'tol',
 'xyl',
 'cres',
 'to2',
 'cro',
 'open',
 'onit',
 'rooh',
 'ro2',
 'ano2',
 'nap',
 'xo2',
 'xpar',
 'isop',
 'isoprd',
 'isopp',
 'isopn',
 'isopo2',
 'api',
 'lim',
 'dms',
 'msa',
 'dmso',
 'dmso2',
 'ch3so2h',
 'ch3sch2oo',
 'ch3so2',
 'ch3so3',
 'ch3so2oo',
 'ch3so2ch2oo',
 'SULFHOX',
 'P',
 'PB']

target_cols = [
    'ccn_001',
     'ccn_003',
     'ccn_006',
     'CHI',
     'CHI_CCN',
     'D_ALPHA',
     'D_GAMMA',
     'D_ALPHA_CCN',
     'D_GAMMA_CCN',
     'PM25'
]

# Reading With Xarray

In [11]:
ds_training = xr.open_dataset("training.nc")

In [23]:
da = ds_training[['ccn_001', 'ccn_003']].sel(
    Time=xr.DataArray([30, 1], dims="z"), 
    south_north=xr.DataArray([10, 19], dims="z"),
    west_east=xr.DataArray([20, 9], dims="z",),
    bottom_top=xr.DataArray([3,23],dims='z')
)

In [37]:
t = da.to_array().to_numpy()

In [52]:
class Indexer:
    def __init__(self, ndims):
        self.ndims = ndims
    def idx4(self, a):
        t, k, j, i = a
        nt, nz, ny, nx = self.ndims
        return ((nz*t + k)*ny + j)*nx + i
    def inverse4(self, ix):
        nt, nz, ny, nx = self.ndims
        total = nz*ny*nx
        t = ix// total
        ix = ix % total
        
        total = ny*nx
        k = ix // total
        ix = ix % total
        
        total = nx
        j = ix//total
        i = ix % total
        return (t,k,j,i)
idxer = Indexer(ds_training['ccn_001'].shape)
import numpy as np
idxer.inverse4( np.array([213,313]) )

(array([0, 0]), array([0, 0]), array([1, 1]), array([ 44, 144]))

In [56]:
isinstance(np.array([213,313]), np.ndarray)

True

In [62]:
class RegDataset:
    def __init__(self,filename,features,target):
        super().__init__()
        self.ncf = xr.open_dataset(filename)
        self.target = target
        self.feat   = features
        self.ixer   = idxer = Indexer(self.ncf['ccn_001'].shape)
        
    def __getitem__(self,idxes):
        assert isinstance(idxes, np.ndarray)
        ix4 = self.ixer.inverse4(idxes)
        tidx = xr.DataArray(ix4[0], dims="z")
        zidx = xr.DataArray(ix4[1], dims="z")
        yidx = xr.DataArray(ix4[2], dims="z")
        xidx = xr.DataArray(ix4[3], dims="z")
        
        da = ds_training[self.feat].sel(
            Time=tidx,
            bottom_top=zidx,
            south_north =yidx,
            west_east=xidx,
        )
        return da.to_array().to_numpy()
        
    def __len__(self):
        return NMAX

In [63]:
ds = RegDataset(wrf_filename, feat_cols, target_cols)
ds

In [70]:
np.prod(ds.ncf['ccn_001'].shape)

139379877

In [80]:
%%timeit
NMAX = 1393798
idxes = np.random.randint(0,NMAX,8)
ds[idxes]

2.9 s ± 290 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [202]:
f

'PM25'

In [203]:
ds.ncf[f][]

<xarray.DataArray 'PM25' (Time: 133, bottom_top: 39, south_north: 159,
                          west_east: 169)>
[139379877 values with dtype=float32]
Coordinates:
    XLAT     (Time, south_north, west_east) float32 ...
    XLONG    (Time, south_north, west_east) float32 ...
Dimensions without coordinates: Time, bottom_top, south_north, west_east
Attributes:
    MemoryOrder:  XYZ
    description:  PM25, total mass
    units:        kg m^-3
    coordinates:  XLONG XLAT XTIME

In [7]:
# Load netcdf file
from netCDF4 import Dataset
import numpy as np
data_dir = './'
wrf_filename = '%straining.nc' % data_dir
ncf = Dataset(wrf_filename, "r", format="NETCDF4")

time_axis = ["".join( ncf.variables['Times'][i].data.astype(str))[-8:] for i in range(133)]
hrs  = [float(time_axis[i].split(":")[0]) for i in range(len(time_axis))]
mins_in_hrs = [float(time_axis[i].split(":")[1])/60 for i in range(len(time_axis))]

timestamp_hrs = np.array([(h+m)/24 for h,m in zip(hrs, mins_in_hrs)])
time_embed = np.cos(timestamp_hrs*np.pi*2)

In [15]:
ds_training[feat_cols[0]][0,...].data.flatten().shape

(1047969,)

In [8]:
!mkdir time_embed_data

In [18]:
for i in range(133):
    tmp = []
    for f in feat_cols:
        tmp.append(ds_training[f][i,...].data.flatten())
    tmp.append(np.ones_like(ds_training[f][i,...].data.flatten())* time_embed[i])
    np.save(f'time_embed_data/t{i}_feat.npy', np.vstack(tmp).transpose())

In [20]:
for i in range(133):
    tmp = []
    for f in target_cols:
        tmp.append(ds_training[f][i,...].data.flatten())
    np.save(f'time_embed_data/t{i}_targ.npy', np.vstack(tmp).transpose())

In [220]:
import pandas as pd
# import seaborn as sns

pd.DataFrame(out, columns=target_cols).corr()

,ccn_001,ccn_003,ccn_006,CHI,CHI_CCN,D_ALPHA,D_GAMMA,D_ALPHA_CCN,D_GAMMA_CCN,PM25
ccn_001,1.000000,0.931942,0.878823,-0.017472,-0.163787,0.343573,0.357511,-0.231737,-0.122894,0.735995
ccn_003,0.931942,1.000000,0.985097,-0.097920,-0.268909,0.433797,0.462196,-0.133836,0.021081,0.661639
ccn_006,0.878823,0.985097,1.000000,-0.129980,-0.312378,0.438712,0.476438,-0.129309,0.048147,0.639433
CHI,-0.017472,-0.097920,-0.129980,1.000000,0.685764,-0.307690,-0.479376,0.067522,-0.213222,0.173920
CHI_CCN,-0.163787,-0.268909,-0.312378,0.685764,1.000000,-0.132114,-0.274026,0.146654,-0.255448,0.035312
D_ALPHA,0.343573,0.433797,0.438712,-0.307690,-0.132114,1.000000,0.976192,0.615468,0.681524,-0.020273
D_GAMMA,0.357511,0.462196,0.476438,-0.479376,-0.274026,0.976192,1.000000,0.521888,0.653417,-0.002380
D_ALPHA_CCN,-0.231737,-0.133836,-0.129309,0.067522,0.146654,0.615468,0.521888,1.000000,0.911623,-0.474397
D_GAMMA_CCN,-0.122894,0.021081,0.048147,-0.213222,-0.255448,0.681524,0.653417,0.911623,1.000000,-0.421686
PM25,0.735995,0.661639,0.639433,0.173920,0.035312,-0.020273,-0.002380,-0.474397,-0.421686,1.000000


In [105]:
ridx = np.random.randint(0,1047969,32)

In [116]:
40*100

4000

In [121]:
from torch.utils.data import TensorDataset
import torch

In [122]:
feats = np.load('t0_feat.npy')
targs = np.load('t0_targ.npy')

In [204]:
feats.shape

(1047969, 100)

In [123]:
tds = TensorDataset( torch.from_numpy(feats), torch.from_numpy(targs) )

In [147]:
means = {}
stds  = {}
mins  = {}
for i, f in enumerate(feat_cols):
    tmp = feats[:,i]
    d = tmp[tmp>0]
    means[f] = np.log(d).mean()
    stds[f]  = np.log(d).std()
    
    mins[f] = np.exp(np.log(d).min())
    
for i, f in enumerate(target_cols):
    tmp = targs[:,i]
    d = tmp[tmp>0]
    means[f] = np.log(d).mean()
    stds[f]  = np.log(d).std()
    
    mins[f] = np.exp(np.log(d).min())

feat_mean_list = torch.from_numpy(np.array([means[f] for f in feat_cols]))
feat_std_list = torch.from_numpy(np.array([stds[f] for f in feat_cols]))
feat_min_list = torch.from_numpy(np.array([mins[f] for f in feat_cols]))

targ_mean_list = torch.from_numpy(np.array([means[f] for f in target_cols]))
targ_std_list = torch.from_numpy(np.array([stds[f] for f in target_cols]))
targ_min_list = torch.from_numpy(np.array([mins[f] for f in target_cols]))

In [148]:
feat_mean_list

tensor([ 18.3412, -22.1450, -22.9682, -34.2544, -67.2179, -35.4954, -47.4209,
        -49.5522, -42.6411, -47.1696, -25.1341, -58.5334, -58.0795, -58.2384,
        -24.7795, -25.5289, -23.3483,   5.5045,  -1.5787,   0.8495,   8.6020,
        -10.4790,  -1.6457, -30.5257, -15.5252,  -3.0493,  -3.1118, -12.9695,
         -9.1049,  -8.4593,  -4.6355,   4.7527, -26.0608, -12.8952,  -8.7799,
         -5.4120,  -1.4715,   4.2199,  -6.5071, -26.5848, -25.2973, -14.3721,
        -26.6392,  -4.1990, -15.8800, -27.3723, -12.4017, -23.0230, -16.1754,
        -18.5365, -12.5285, -15.0489,  -2.4228, -27.5092, -28.3856, -23.7684,
        -27.4414, -17.4906, -36.7175, -36.6878, -36.5561, -19.5772, -19.7742,
        -11.2861,  -8.3644, -27.7195, -29.6143, -26.8391, -28.1484, -28.7525,
        -29.5755, -31.3890, -28.1026, -20.8438, -19.5698, -23.1247, -26.2996,
        -25.8858, -18.8093, -20.9491, -26.4075, -14.3241, -28.3505, -31.5561,
        -25.4605, -36.8598, -36.8948, -36.8413, -36.8360, -36.84

In [133]:
(tds[0][0].log() - feat_mean_list) / feat_std_list

tensor([ 3.1084,  2.0410,  3.0326,    -inf,    -inf,    -inf,    -inf,    -inf,
           -inf,    -inf,  2.2685,    -inf,    -inf,    -inf,  1.1498,  1.5756,
           -inf,  1.2864,  2.1053, -1.2878, -4.0542,  0.7837, -0.1046, -0.6235,
         0.9626, -0.0274,  0.4711,  0.8871, -0.8522,  0.4350, -1.0270, -1.0865,
        -1.0651, -0.8522,  0.0780,  0.2847,  0.7023,  0.7970,  2.0737, -0.7312,
         0.1211,  0.9427,  0.8729,  1.4956,  0.3374, -0.7101,  0.6330,  0.6013,
         0.7195,  0.6172,  0.5611,  0.9361,  0.3940, -0.8264, -0.7747,  0.3749,
        -0.6534,  1.1781, -0.4706, -0.5042, -0.5629,  0.6995,  0.8750,  1.4093,
         0.8740, -0.6975, -0.6156, -0.8115, -0.7560, -0.7437, -0.7087, -0.6198,
        -0.7138,  0.8024,  0.9217,  0.8628,  0.8739,  0.8081,  0.9104,  0.8516,
         1.5212,  0.9953,  1.5792,  1.5050,  1.3188,  0.4801,  0.4765, -0.0823,
        -0.0643,  0.0084, -0.0473, -0.0479,  0.0249,  0.0251,  0.0233,  0.0251,
         0.0244, -0.6073,  1.2655,  1.30

In [150]:
X = tds[0][0]

dl =torch.utils.data.DataLoader(tds, batch_size=32, shuffle=True)



In [152]:
X, y = next(iter(dl))

In [157]:
feat_mean_list.shape, X.shape

(torch.Size([100]), torch.Size([32, 100]))

In [185]:
padded = torch.where(X < feat_min_list.unsqueeze(0), feat_min_list.unsqueeze(0), X)

Xpad_norm = (padded.log() - feat_mean_list)/ feat_std_list

In [186]:
padded[7]

tensor([7.0935e+07, 1.3758e-10, 1.1464e-10, 1.1185e-22, 8.5731e-36, 1.7847e-26,
        2.3761e-29, 2.2701e-29, 3.3750e-29, 2.1600e-29, 4.2616e-12, 2.4669e-29,
        2.5750e-29, 2.5750e-29, 1.1290e-11, 7.3883e-12, 3.1711e-22, 2.6921e+02,
        3.3047e-01, 1.3243e+00, 4.5590e+03, 1.0000e-16, 5.2718e-02, 1.0000e-16,
        1.1329e-14, 9.8469e-03, 9.8521e-03, 7.3207e-07, 1.6618e-07, 1.0000e-16,
        2.0205e-03, 3.7799e+01, 1.0000e-16, 1.0000e-16, 1.0000e-16, 1.0000e-16,
        1.6495e+00, 9.1820e+01, 1.7016e-04, 1.0000e-16, 1.0000e-16, 1.0000e-16,
        1.0000e-16, 1.8324e-01, 1.0000e-16, 1.0000e-16, 1.0000e-16, 1.0000e-16,
        1.0000e-16, 1.0000e-16, 1.0000e-16, 1.0000e-16, 1.1148e-01, 1.0000e-16,
        1.0000e-16, 1.0000e-16, 1.0000e-16, 1.0000e-16, 1.0000e-16, 1.0000e-16,
        1.0000e-16, 1.0000e-16, 1.0000e-16, 2.7423e-04, 5.6835e-05, 1.0000e-16,
        1.0000e-16, 1.0000e-16, 1.0000e-16, 1.0000e-16, 1.0000e-16, 1.0000e-16,
        1.0000e-16, 1.0000e-16, 1.0000e-

In [187]:
Xpad_norm[7]

tensor([-3.7181e-01, -5.2463e-01,  8.1649e-02, -4.7173e+00, -1.2661e+00,
        -2.3137e+00, -1.4520e+00, -1.4442e+00, -1.8356e+00, -1.3992e+00,
        -1.4333e+00, -4.6390e+00, -3.8346e+00, -4.1268e+00, -4.2194e-01,
        -1.0362e-01, -2.0233e+01,  7.4510e-01,  6.7880e-01, -7.0241e-01,
        -1.3670e-01, -7.1179e+00, -9.5014e-01, -6.2355e-01, -1.6974e+00,
        -1.2110e+00, -1.1016e+00, -7.7145e-01, -1.4573e+00, -8.8401e+00,
        -9.8024e-01, -8.8463e-01, -4.5833e+00, -6.5522e+00, -9.2605e+00,
        -9.1743e+00,  1.0297e+00,  4.9452e-01, -1.0048e+00, -7.3124e-01,
        -9.3434e-01, -3.3148e+00, -8.6796e-01,  1.2422e+00, -1.7256e+00,
        -7.1012e-01, -3.0034e+00, -1.1931e+00, -1.7161e+00, -1.5444e+00,
        -2.8770e+00, -3.1739e+00,  1.8731e-01, -8.2638e-01, -7.7474e-01,
        -1.1076e+00, -8.1657e-01, -2.2995e+00, -4.7060e-01, -5.0417e-01,
        -5.6293e-01, -1.2515e+00, -1.2994e+00,  7.4459e-01, -2.4016e-01,
        -6.9752e-01, -6.1556e-01, -8.1152e-01, -7.5

In [188]:
import torch.nn as nn
from collections import OrderedDict

In [200]:
class SimpleMLP(torch.nn.Module):
    def __init__(self, ninputs=100, nhidden=256, nouts=10, activations='ELU'):
        super().__init__()
        self.layers = OrderedDict([
            ('linear1', nn.Linear(ninputs,nhidden)),
            ('act1', nn.ELU()),
            ('linear2', nn.Linear(nhidden,nhidden)),
            ('act2', nn.ELU()),
            ('linear3', nn.Linear(nhidden,nhidden)),
            ('act3', nn.ELU()),
            ('linear4', nn.Linear(nhidden,nouts)),
        ])
        self.net = torch.nn.Sequential(self.layers)
    
    def forward(self, x):
        return self.net(x)
        

In [224]:
len(dl)

32750

In [230]:
dl =torch.utils.data.DataLoader(tds, batch_size=512, shuffle=True)

net = SimpleMLP()

def loss_fn(ypred, y):
    return ((ypred-y)**2).mean()

optim = torch.optim.Adam(net.parameters())

total_loss = 0
count = 0
for X, y in dl:
    optim.zero_grad()

    padded = torch.where(X < feat_min_list.unsqueeze(0), feat_min_list.unsqueeze(0), X)
    Xpad_norm = (padded.log() - feat_mean_list)/ feat_std_list
    
    padded = torch.where(y < targ_min_list.unsqueeze(0), targ_min_list.unsqueeze(0), y)
    ypad_norm = (padded.log() - targ_mean_list)/ targ_std_list
    
    ypred = net(Xpad_norm)
    
    loss = loss_fn(ypred, ypad_norm)
    loss.backward()
    optim.step()
    
    total_loss += loss.item()
    
    if count % 100 == 0:
        print(count / len(dl))
    count += 1
    
print(total_loss/ len(dl))

0.0
0.048851978505129456
0.09770395701025891
0.14655593551538837
0.19540791402051783
0.24425989252564728
0.29311187103077674
0.3419638495359062
0.39081582804103565
0.4396678065461651
0.48851978505129456
0.537371763556424
0.5862237420615535
0.6350757205666829
0.6839276990718124
0.7327796775769418
0.7816316560820713
0.8304836345872008
0.8793356130923302
0.9281875915974597
0.9770395701025891
0.04482922329224781


In [231]:
total_loss

91.76542007923126

In [229]:
loss

tensor(0.0180, grad_fn=<MeanBackward0>)

In [233]:
count / len(dl)

1.0

In [234]:

from tkinter import *
from tkinter import filedialog as fd
import os
from PIL import Image
from tkinter import messagebox